<a href="https://colab.research.google.com/github/4may/SuperResolutionSurvey/blob/master/FSRCNN_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fast Super-Resolution Convolutional Network(FSRCNN)

[OSSのソース](https://github.com/GeorgeSeif/FSRCNN-Keras)を参考に実装してみる。

* data
  * trainingデータとして、291種類の画像データを使用している。
    * 明確なデータが確認できなかったが、おそらく91(.bmp)+200(.jpg)で構成されている
    * matlabのプログラムでデータを加工しているのだが、拡張子が.bmpと.jpgの二つある
      * [91](https://www.kaggle.com/ll01dm/t91-image-dataset):.bmpファイル。
      * [200](https://www2.eecs.berkeley.edu/Research/Projects/CS/vision/bsds/BSDS300/html/dataset/images.html):.jpgファイル。確度そこそこ。train/testで分かれていて、それぞれ200と100枚。
  * testデータとして、5種類の画像データを使用している。
    * こちらは、明確にSet5との記載あり。
* アーキテクチャ
  * コスパの良いFSRCNN(56, 16, 4)が採用されている。

## Data Augmentation

[matlabのソース](https://github.com/GeorgeSeif/FSRCNN-Keras/blob/master/data/aug_train.m)を参考に、291種類の画像データに対してdata augmentationを行う。

In [0]:
!unzip "291.zip"

Archive:  291.zip
   creating: 291/
  inflating: 291/157036.jpg          
  inflating: __MACOSX/291/._157036.jpg  
  inflating: 291/187029.jpg          
  inflating: __MACOSX/291/._187029.jpg  
  inflating: 291/71046.jpg           
  inflating: __MACOSX/291/._71046.jpg  
  inflating: 291/299091.jpg          
  inflating: __MACOSX/291/._299091.jpg  
  inflating: 291/tt26.bmp            
  inflating: __MACOSX/291/._tt26.bmp  
  inflating: 291/242078.jpg          
  inflating: __MACOSX/291/._242078.jpg  
  inflating: 291/t53.bmp             
  inflating: __MACOSX/291/._t53.bmp  
  inflating: 291/t47.bmp             
  inflating: __MACOSX/291/._t47.bmp  
  inflating: 291/232038.jpg          
  inflating: __MACOSX/291/._232038.jpg  
  inflating: 291/173036.jpg          
  inflating: __MACOSX/291/._173036.jpg  
  inflating: 291/198054.jpg          
  inflating: __MACOSX/291/._198054.jpg  
  inflating: 291/t46.bmp             
  inflating: __MACOSX/291/._t46.bmp  
  inflating: 291/388016.jpg 

In [0]:
import cv2
import numpy as np
import os
from glob import glob
import math
import matplotlib.pyplot as plt
import imageio

291種類のファイルのパスを取得する。

In [0]:
#291種類の画像データが格納されているフォルダ
TRAIN_SOURCE_DIR = '291'

#data augmentationをした画像を保存するフォルダ
TRAIN_TARGET_DIR = 'train'

sources = glob("{}/*.jpg".format(TRAIN_SOURCE_DIR)) + glob("{}/*.bmp".format(TRAIN_SOURCE_DIR))

In [0]:
sources[:5]

['291/187071.jpg',
 '291/48055.jpg',
 '291/2092.jpg',
 '291/20008.jpg',
 '291/43070.jpg']

In [0]:
sources[286:]

['291/t2.bmp', '291/t17.bmp', '291/tt27.bmp', '291/t5.bmp', '291/tt25.bmp']

In [0]:
len(sources)

291

各ファイルパスから得られる画像に対して、data augmentationを行う。

In [0]:
#freeimageをdownloadしておかないとエラーとなる。
imageio.plugins.freeimage.download()

#保存先ディレクトリの作成
if os.path.isdir(TRAIN_TARGET_DIR) == False:
  os.mkdir(TRAIN_TARGET_DIR)

#パッチの総数
patch_count = 0
#処理した画像の枚数
img_num = 0
for source in sources:
  #パスから画像をロード
  img_raw = cv2.imread(source)
  img_raw = cv2.cvtColor(img_raw, cv2.COLOR_BGR2YCrCb)
  
  #ピクセル値を0.0~1.0の間に正規化
  #https://ja.coder.work/so/python/385751
  img_raw = cv2.normalize(img_raw.astype(float), None, 0.0, 1.0, cv2.NORM_MINMAX)

  height, width, _ = img_raw.shape
  size = (width, height)

  #クロッピング
  img_raw = img_raw[1:(height-np.mod(height, 12)), 0:(width-np.mod(width, 12))]

  img_shape = img_raw.shape
  patch_size = 32
  stride = 32
  x_size = math.floor((width - patch_size)/stride + 1)
  y_size = math.floor((height - patch_size)/stride + 1)

  #data augmentation
  for x in range(x_size):
    for y in range(y_size):
      x_coord = x * stride
      y_coord = y * stride

      #パッチその１(クリッピングのみ)
      patch_name = os.path.join(TRAIN_TARGET_DIR, str(patch_count))
      patch = img_raw[y_coord+1:y_coord+patch_size, x_coord+1:x_coord+patch_size]
      imageio.imwrite('{}.exr'.format(patch_name), patch.astype('float32'), format='EXR-FI')
      ph, pw, _ = patch.shape
      patch_2 = cv2.resize(patch, (ph // 2, pw // 2), cv2.INTER_CUBIC)
      imageio.imwrite('{}_2.exr'.format(patch_name), patch_2.astype('float32'), format='EXR-FI')
      patch_3 = cv2.resize(patch, (ph // 3, pw // 3), cv2.INTER_CUBIC)
      imageio.imwrite('{}_3.exr'.format(patch_name), patch_3.astype('float32'), format='EXR-FI')
      patch_4 = cv2.resize(patch, (ph // 4, pw // 4), cv2.INTER_CUBIC)
      imageio.imwrite('{}_4.exr'.format(patch_name), patch_4.astype('float32'), format='EXR-FI')

      patch_count += 1

      #パッチその2(クリッピング+90°回転)
      patch_name = os.path.join(TRAIN_TARGET_DIR, str(patch_count))
      patch = img_raw[y_coord+1:y_coord+patch_size, x_coord+1:x_coord+patch_size]
      patch = np.rot90(patch, k=1)
      imageio.imwrite('{}.exr'.format(patch_name), patch.astype('float32'), format='EXR-FI')
      ph, pw, _ = patch.shape
      patch_2 = cv2.resize(patch, (ph // 2, pw // 2), cv2.INTER_CUBIC)
      imageio.imwrite('{}_2.exr'.format(patch_name), patch_2.astype('float32'), format='EXR-FI')
      patch_3 = cv2.resize(patch, (ph // 3, pw // 3), cv2.INTER_CUBIC)
      imageio.imwrite('{}_3.exr'.format(patch_name), patch_3.astype('float32'), format='EXR-FI')
      patch_4 = cv2.resize(patch, (ph // 4, pw // 4), cv2.INTER_CUBIC)
      imageio.imwrite('{}_4.exr'.format(patch_name), patch_4.astype('float32'), format='EXR-FI')

      patch_count += 1

      #パッチその3(クリッピング+左右反転)
      patch_name = os.path.join(TRAIN_TARGET_DIR, str(patch_count))
      patch = img_raw[y_coord+1:y_coord+patch_size, x_coord+1:x_coord+patch_size]
      patch = np.fliplr(patch)
      imageio.imwrite('{}.exr'.format(patch_name), patch.astype('float32'), format='EXR-FI')
      ph, pw, _ = patch.shape
      patch_2 = cv2.resize(patch, (ph // 2, pw // 2), cv2.INTER_CUBIC)
      imageio.imwrite('{}_2.exr'.format(patch_name), patch_2.astype('float32'), format='EXR-FI')
      patch_3 = cv2.resize(patch, (ph // 3, pw // 3), cv2.INTER_CUBIC)
      imageio.imwrite('{}_3.exr'.format(patch_name), patch_3.astype('float32'), format='EXR-FI')
      patch_4 = cv2.resize(patch, (ph // 4, pw // 4), cv2.INTER_CUBIC)
      imageio.imwrite('{}_4.exr'.format(patch_name), patch_4.astype('float32'), format='EXR-FI')

      patch_count += 1

      #パッチその4(クリッピング+左右反転+90°回転)
      patch_name = os.path.join(TRAIN_TARGET_DIR, str(patch_count))
      patch = img_raw[y_coord+1:y_coord+patch_size, x_coord+1:x_coord+patch_size]
      patch = np.fliplr(patch)
      patch = np.rot90(patch, k=1)
      imageio.imwrite('{}.exr'.format(patch_name), patch.astype('float32'), format='EXR-FI')
      ph, pw, _ = patch.shape
      patch_2 = cv2.resize(patch, (ph // 2, pw // 2), cv2.INTER_CUBIC)
      imageio.imwrite('{}_2.exr'.format(patch_name), patch_2.astype('float32'), format='EXR-FI')
      patch_3 = cv2.resize(patch, (ph // 3, pw // 3), cv2.INTER_CUBIC)
      imageio.imwrite('{}_3.exr'.format(patch_name), patch_3.astype('float32'), format='EXR-FI')
      patch_4 = cv2.resize(patch, (ph // 4, pw // 4), cv2.INTER_CUBIC)
      imageio.imwrite('{}_4.exr'.format(patch_name), patch_4.astype('float32'), format='EXR-FI')

      patch_count += 1
  
  img_num += 1
  print(img_num)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


float32だと流石に書き込みに時間がかかる。10分ぐらいかかった。

data augmentationした画像が保存できているかどうか確認する。

In [0]:
img = imageio.imread('train/0.exr')

In [0]:
img[0, 0]

Array([0.36743164, 0.42456055, 0.57958984], dtype=float32)

In [0]:
type(img[0, 0, 0])

numpy.float32

続いて、testデータもdata augmentationする。testデータのdata augmentationの手順は、traindデータのそれとほぼ一緒。testデータのdata augmentationでは、パッチの種類が１種類である点だけが異なる。

まずは、test画像のファイルパスを取得する。

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
!unzip 'Set5.zip'

Archive:  Set5.zip
   creating: Set5/
  inflating: __MACOSX/._Set5         
  inflating: Set5/woman_GT.bmp       
  inflating: __MACOSX/Set5/._woman_GT.bmp  
  inflating: Set5/butterfly_GT.bmp   
  inflating: __MACOSX/Set5/._butterfly_GT.bmp  
  inflating: Set5/bird_GT.bmp        
  inflating: __MACOSX/Set5/._bird_GT.bmp  
  inflating: Set5/head_GT.bmp        
  inflating: __MACOSX/Set5/._head_GT.bmp  
  inflating: Set5/baby_GT.bmp        
  inflating: __MACOSX/Set5/._baby_GT.bmp  


In [0]:
TEST_SOURCE_DIR = 'Set5'
TEST_TARGET_DIR = 'test'

test_sources = glob('{}/*.bmp'.format(TEST_SOURCE_DIR))

In [0]:
len(test_sources)

5

In [0]:
test_sources

['Set5/head_GT.bmp',
 'Set5/butterfly_GT.bmp',
 'Set5/baby_GT.bmp',
 'Set5/woman_GT.bmp',
 'Set5/bird_GT.bmp']

取得した画像ファイルのパスを使って、data augmentationする。

In [0]:
#data augmentationしたtest画像の保存先ディレクトリを作成
if os.path.isdir(TEST_TARGET_DIR) == False:
  os.mkdir(TEST_TARGET_DIR)

#data augmentation
path_count = 0
img_count = 0

for source in test_sources:
  #YCrCb空間で読み取り
  img_raw = cv2.imread(source)
  img_raw = cv2.cvtColor(img_raw, cv2.COLOR_BGR2YCrCb)
  img_raw = img_raw[:, :, 1]
  
  #正規化
  img_raw = cv2.normalize(img_raw.astype(float), None, 0.0, 1.0, cv2.NORM_MINMAX)

  #クロッピング
  height, width = img_raw.shape
  img_raw = img_raw[1:(height-np.mod(height, 12)), 0:(width-np.mod(width, 12))]

  #パッチ化
  patch_size = 32
  stride = 14
  x_size = math.floor((width - patch_size)/stride) + 1
  y_size = math.floor((height - patch_size)/stride) + 1
  for x in range(x_size):
    for y in range(y_size):
      x_coord = x * stride
      y_coord = y * stride

      patch_name = os.path.join(TEST_TARGET_DIR, str(path_count))
      patch = img_raw[y_coord+1:y_coord+patch_size, x_coord+1:x_coord+patch_size]
      #ピクセル値をfloat32のまま保存しておきたい。そのため、imageio.imwrite関数を使う。
      #ポイントは、拡張子として.exrを指定することと、formatとして'EXR-FI'を指定すること。
      #なお、ファイルを呼び出すときは、imageio.imread('hoge.exr')とすれば良い。
      #また、patchのピクセル値の型はfloat64であるが、float64は保存できないため、float32にする
      imageio.imwrite('{}.exr'.format(patch_name), patch.astype('float32'), format='EXR-FI')
      ph, pw = patch.shape
      patch_2 = cv2.resize(patch, (ph // 2, pw // 2), cv2.INTER_CUBIC)
      imageio.imwrite('{}_2.exr'.format(patch_name), patch_2.astype('float32'), format='EXR-FI')
      patch_3 = cv2.resize(patch, (ph // 3, pw // 3), cv2.INTER_CUBIC)
      imageio.imwrite('{}_3.exr'.format(patch_name), patch_3.astype('float32'), format='EXR-FI')
      patch_4 = cv2.resize(patch, (ph // 4, pw // 4), cv2.INTER_CUBIC)
      imageio.imwrite('{}_4.exr'.format(patch_name), patch_4.astype('float32'), format='EXR-FI')

      path_count += 1
  
  img_count += 1
  print(img_count)
  break

1


float32として保存できていることを確認する。

In [0]:
img = imageio.imread('test/0.exr')

In [0]:
img[0, 0]

0.5576172

In [0]:
type(img[0, 0])

numpy.float32